# 제32회 ADP 실기 대비 - 핵심만 요약한 통계와 머신러닝 파이썬 코드북

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# 시각화 설정
plt.rcParams['font.family'] = 'Malgun Gothic' # 한글 폰트 설정
plt.rcParams['axes.unicode_minus'] = False # 마이너스 부호 설정

# Colab 한글 폰트 설정
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf
# plt.rcParams['font.family'] = 'NanumBarunGothic'

# 7장. 공분산과 상관계수

## 7-1. 공분산(Covariance)
- 공분산은 X변수가 얼마만큼 변할 때 다른 변수 Y가 얼만큼 변하는지를 나타내는 값이다.
- 공분산이 양수이면 X와 Y는 양의 선형관계를 갖고, 음수이면 음의 선형관계를 가지며, 두 변수가 서로 독립이면 공분산은 0이 된다.
- 한편, 공분산이 0이라고 해도 두 변수가 반드시 서로 독립인 것은 아니다. 비선형적 연관성이 있을 수 있기 때문이다.

In [2]:
X = np.array([4, 6, 6, 8, 8, 9, 9, 10, 12, 12])
Y = np.array([39, 42, 45, 47, 50, 50, 52, 55, 57, 60])

Xm = np.mean(X)
Ym = np.mean(Y)
n = len(X)

# 공분산을 계산하는 두 가지 방법
cov1 = (np.sum(X * Y) - n * Xm * Ym) / (n-1)
cov2 = sum((X - Xm) * (Y - Ym)) / (n-1)

print(f"{round(cov1, 2)} {round(cov2, 2)}")

16.8 16.8


In [3]:
cov3 = np.cov(X, Y) # numpy는 공분산 행렬을 반환해줌

print(pd.DataFrame(cov3, columns=['X', 'Y'], index=['X', 'Y']))
print(f"X-Y의 공분산: {round(cov3[0][1], 2)}")

           X          Y
X   6.711111  16.800000
Y  16.800000  44.011111
X-Y의 공분산: 16.8


## 7-2. 상관계수(Correlation coefficient)
- 공분산은 변수의 측정 단위의 영향을 받기 때문에 그 값 자체로는 연관성이 얼마나 높은지 가늠하기 어렵다.
- 이런 단점을 해결하기 위해 공분산을 각 변수의 표준편차로 나눈 상관계수를 사용한다.
- 상관계수는 변수의 특성에 따라 피어슨 상관계수, 스피어만 순위상관계수, 켄달의 타우가 있다.
- 이 외에도 명목척도로 된 범주형 데이터 간의 연관성을 나타내는 크라메르의 연관계수도 있다.

### 피어슨 상관계수(Pearson correlation coefficient)
- 피어슨 상관계수는 표본상관계수, 적률상관계수라고도 한다.
- 연속형 변수(등간척도, 비율척도)로 측정된 변수들 사이의 선형관계를 나타내며, 확률분포가 정규분포를 따른다고 가정한다.
- 상관계수의 범위는 -1 ~ 1이며, -1에 가까울수록 음의 상관성, 1에 가까울수록 양의 상관성, 0에 가까울수록 상관관계가 없다고 본다.
- Z분포를 사용하여 모상관계수의 신뢰구간을 추정할 수 있고, t분포 또는 Z분포를 사용하여 모상관계수($p$)가 0인지 아닌지 여부를 검정(무상관 검정)할 수 있다.
- 표본상관계수가 아닌 모집단의 상관계수는 표본공분산과 표본편차 대신 모공분산과 모표준편차를 대입하여 계산한다.

In [4]:
# 피어슨 상관계수 (= 표본상관계수)
X = np.array([4, 6, 6, 8, 8, 9, 9, 10, 12, 12])
Y = np.array([39, 42, 45, 47, 50, 50, 52, 55, 57, 60])

# 표본상관계수 계산하는 두 가지 방법
Xm = np.mean(X)
Ym = np.mean(Y)
n = len(X)

corr1 = (np.sum(X * Y) - n * Xm * Ym) / np.sqrt(sum((X - Xm)**2) * sum((Y - Ym)**2))

Sx = np.sqrt(sum((X - Xm)**2) / (n-1)) # X 표본표준편차
Sy = np.sqrt(sum((Y - Ym)**2) / (n-1)) # Y 표본표준편차
cov1 = (np.sum(X * Y) - n * Xm * Ym) / (n-1) # X, Y의 공분산
corr2 = cov1 / (Sx * Sy) # 공분산을 X, Y의 표본표준편차들로 나누어서 구한다.
print(f"{round(corr1, 2)} {round(corr2, 2)}")

0.98 0.98


In [5]:
corr3 = np.corrcoef(X, Y)[0][1]

from scipy.stats import pearsonr
corr4, pval = pearsonr(X, Y) # 피어슨 상관계수와 유의확률을 반환해 줌

print(f"{round(corr3, 2)}, {round(corr4, 2)} (p-value: {round(pval, 2)})")

0.98, 0.98 (p-value: 0.0)


In [6]:
# 모상관계수의 점추정량과 신뢰구간 계산: Z분포 사용
from scipy.stats import norm

conf_a = 0.05
conf_z = norm.ppf(1 - conf_a / 2) # or 1.96

r1 = corr1 # 위에서 계산한 표본상관계수 corr1
n = len(X) # 표본의 개수
a = 1/2 * np.log((1+r1) / (1-r1)) - conf_z / np.sqrt(n-3)
b = 1/2 * np.log((1+r1) / (1-r1)) + conf_z / np.sqrt(n-3)

conf1 = (np.exp(2*a)-1) / (np.exp(2*a)+1) # = np.tanh(a)
conf2 = (np.exp(2*b)-1) / (np.exp(2*b)+1) # = np.tanh(b)

print("[추정]")
print(f" 점   추정량: {round(r1, 3)}")
print(f" 구간 추정량: {round(conf1, 3)} ~ {round(conf2, 3)}")

[추정]
 점   추정량: 0.978
 구간 추정량: 0.905 ~ 0.995


In [7]:
# 모상관계수의 가설검정: t분포를 사용하며, 표본이 충분히 큰 경우에는 Z분포 사용
# 귀무가설(H0): 모상관계수는 0이다.
# 대립가설(H1): 모상관계수는 0이 아니다. (통계적으로 유의하다)
from scipy.stats import t

r0 = 0 # 귀무가설의 모상관계수
r1 = corr1 # 위에서 계산한 표본상관계수 corr1
n = len(X) # 표본의 개수
df = n-2 # 자유도

tstat = np.sqrt(df) * (r1 - r0) / np.sqrt(1 - r1**2) # 검정통계량 t
# zstat = 1/2 * np.log((1+r1) / (1-r1)) # 검정통계량 Z
test_a = 0.05

sp = (1- t.cdf(np.abs(tstat), df)) * 2 # 양측검정
cv = t.ppf(1 - test_a / 2, df)
cv = f"+/- {round(cv, 3)}"

print("[검정]")
print(f" 임계값: {cv}, 검정통계량: {round(tstat, 3)}")
print(f" 유의수준: {round(test_a, 3)}, 유의확률: {round(sp, 3)}")

[검정]
 임계값: +/- 2.306, 검정통계량: 13.117
 유의수준: 0.05, 유의확률: 0.0


In [8]:
from scipy.stats import pearsonr
corr, p = pearsonr(X, Y)

# 검정결과 대립가설을 채택하여 모상관계수가 0이 아니기 때문에 구해진 상관계수는 통계적으로 유의하다는 결론을 얻었다.
print(f"상관계수: {round(corr, 3)}, p-value: {round(p, 3)}") 

상관계수: 0.978, p-value: 0.0


### 스피어만 순위상관계수(Spearman's Rank correlation coefficient)
- 수치형 관측치의 분포가 극단적인 분포를 보이거나, 관측치가 서열척도로 되어 있을 때 스피어만의 순위상관계수로 변수들의 상관관계를 나타낼 수 있다.
- 확률분포에 대한 정보가 없어도 되는 비모수적 상관분석이다.
- 순위로 된 데이터 내에 같은 순위가 없다면, 스피어만 순위상관계수는 이 순위 값으로 구한 피어슨 상관계수와 일치한다.
- 스피어만 순위상관계수의 범위는 -1 ~ 1이며, -1에 가까울수록 음의 상관성, 1에 가까울수록 양의 상관성, 0에 가까울수록 상관관계가 없다고 본다.

In [9]:
# 표본 내 같은 순위가 없는 경우
X = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
Y = np.array([4, 3, 5, 2, 6, 1, 10, 7, 8, 9])

rtable = pd.DataFrame({"X":X, "Y":Y})
rtable['Xr'] = rtable['X'].rank(ascending=False)
rtable['Yr'] = rtable['Y'].rank(ascending=False)
corr1 = 1 - 6 * sum((rtable['Xr'] - rtable['Yr']) ** 2) / len(X) / (len(X)**2-1)

print(f"표본 내 동일한 데이터가 없는 경우: {round(corr1, 3)}")

표본 내 동일한 데이터가 없는 경우: 0.661


In [10]:
from scipy.stats import spearmanr

corr2, p = spearmanr(X, Y)
print(f"표본 내 동일한 데이터가 없는 경우: {round(corr2, 3)}, p-value: {round(p, 3)}")

표본 내 동일한 데이터가 없는 경우: 0.661, p-value: 0.038


In [11]:
# 표본 내 같은 순위가 있는 경우
X2 = np.array([10, 9, 9, 8, 8, 8])
Y2 = np.array([6, 7, 8, 8, 7, 6.5])

rtable = pd.DataFrame({"X2":X2, 'Y2':Y2})
rtable['X2r'] = rtable['X2'].rank(ascending=False)
rtable['Y2r'] = rtable['Y2'].rank(ascending=False)

# C: 같은 순위에 포함된 데이터 개수
# Clist: C들이 포함된 리스트
Cxlist = pd.DataFrame(rtable['X2r'].value_counts()).query("X2r >= 2").values.ravel()
Cylist = pd.DataFrame(rtable['Y2r'].value_counts()).query("Y2r >= 2").values.ravel()

# Cx와 Cy 구하기
Cx = 0
Cy = 0

for cx, cy in zip(Cxlist, Cylist):
    Cx += cx*(cx**2-1)
    Cy += cy*(cy**2-1)

# Tx와 Ty 구하기
Tx = 1/12 * (len(X2)*(len(X2)**2-1) - Cx)
Ty = 1/12 * (len(Y2)*(len(Y2)**2-1) - Cy)

corr3 = (Tx+Ty-sum((rtable['X2r']-rtable['Y2r'])**2)) / (2* np.sqrt(Tx*Ty))
print(f"표본 내 동일한 데이터가 있는 경우 {round(corr3, 3)}")

표본 내 동일한 데이터가 있는 경우 -0.318


In [12]:
from scipy.stats import spearmanr

# 유의수준 0.05 기준으로 해당 상관계수는 통계적으로 유의하다고 보기 어렵다.
corr4, p = spearmanr(X2, Y2)
print(f"표본 내 동일한 데이터가 있는 경우 {round(corr4, 3)}, p-value: {round(p, 3)}".format(corr4, p))

표본 내 동일한 데이터가 있는 경우 -0.318, p-value: 0.539


### 켄달의 타우(Kendall's Tau)
- 수치형 관측치의 분포가 극단적인 분포를 보이거나, 관측치가 서열척도로 되어있을 때 사용하는 비모수적 상관계수이다.
- 상관계수의 범위는 -1 ~ 1이며, -1에 가까울수록 음의 상관성, 1에 가까울수록 양의 상관성, 0에 가까울수록 상관관계가 없다고 본다.

In [13]:
# 표본 내 같은 순위가 없는 경우
def combi(x, y):
    from math import factorial as fact
    return fact(x) / fact(x-y) / fact(y)

X = np.array([80, 70, 60, 50, 95, 85, 77, 55, 65])
Y = np.array([6, 3, 2, 1, 8, 9, 7, 4,5])

One = 0
mOne = 0
for i in range(0, len(X)):
    for j in range(i, len(X)):
        if (X[i] - X[j]) * (Y[i] - Y[j]) > 0:
            One += 1
        elif (X[i] - X[j]) * (Y[i] - Y[j]) < 0:
            mOne += 1

corr1 = (1 * One -1 * mOne) / combi(len(X), 2)
print(f"표본 내 동일한 데이터가 없는 경우 {round(corr1, 3)}")

표본 내 동일한 데이터가 없는 경우 0.722


In [14]:
from scipy.stats import kendalltau

corr2, p = kendalltau(X, Y)
print(f"표본 내 동일한 데이터가 없는 경우 {round(corr2, 3)}, p-value: {round(p, 3)}")

표본 내 동일한 데이터가 없는 경우 0.722, p-value: 0.006


In [15]:
# 표본 내 같은 순위가 있는 경우
def combi(x, y):
    from math import factorial as fact
    return fact(x) / fact(x-y) / fact(y)

X2 = np.array([10, 9, 9, 8, 8, 8])
Y2 = np.array([6, 7, 8, 8, 7, 6.5])

One = 0
mOne = 0
for i in range(0, len(X2)):
    for j in range(i, len(X2)):
        if (X2[i] - X2[j]) * (Y2[i] - Y2[j]) > 0:
            One += 1
        elif (X2[i] - X2[j]) * (Y2[i] - Y2[j]) < 0:
            mOne += 1
            
#Cx, Cy:같은 순위에 포함된 데이터 개수
rtable = pd.DataFrame({"X2":X2, 'Y2':Y2})
rtable['X2r'] = rtable['X2'].rank(ascending=False)
rtable['Y2r'] = rtable['Y2'].rank(ascending=False)

#Clist: C들이 포함된 리스트
Cxlist = pd.DataFrame(rtable['X2r'].value_counts()).query("X2r>=2").values.ravel()
Cylist = pd.DataFrame(rtable['Y2r'].value_counts()).query("Y2r>=2").values.ravel()

#Cx와 Cy 구하기
Cx = 0
Cy = 0
for cx, cy in zip(Cxlist, Cylist):
    Cx += 1 / 2 * cx * (cx-1)
    Cy += 1 / 2 * cy * (cy-1)

#Tx와 Ty 구하기
Tx = combi(len(X2), 2) - Cx
Ty = combi(len(X2), 2) - Cy
    
corr3 = (1 * One -1 * mOne) / np.sqrt(Tx * Ty)
print(f"표본 내 동일한 데이터가 있는 경우 {round(corr3, 3)}")

표본 내 동일한 데이터가 있는 경우 -0.251


In [16]:
from scipy.stats import kendalltau

corr4, p = kendalltau(X2, Y2)
print(f"표본 내 동일한 데이터가 있는 경우: {round(corr4, 3)}, p-value: {round(p, 3)}")

표본 내 동일한 데이터가 있는 경우: -0.251, p-value: 0.524


### 크라메르의 연관계수 (Cramer's coefficient of association)
- 크라메르의 연관계수는 범주형 데이터의 교차표를 통해 카이제곱 독립성 검정의 효과의 크기를 측정한다.
- 즉, 두 범주형 데이터가 얼마나 연관성이 있는지를 나타낸다.
- 연관계수의 범위는 0~1이다. 0.2 이하인 경우 연관성이 약하고, 0.6 이상인 경우 연관성이 높다고 본다.

In [17]:
table = pd.DataFrame({
    "성별" : ['남자', '여자'],
    '안경O' : [10, 30],
    '안경X' : [40, 20]
}).set_index('성별')

table

,안경O,안경X
성별,,
남자,10,40
여자,30,20


In [18]:
n = table.sum().sum() # 전체 데이터 개수
exp = []

r = table.sum(axis=1).values
c = table.sum(axis=0).values

for R in r:
    for C in c:
        exp.append(R * C / n)
        
obs = table.values.ravel()
chistat = np.sum((obs - exp)**2 / exp)

k, l = table.shape[0], table.shape[1] # 각 변수의 범주의 개수
V = np.sqrt(chistat / (np.minimum(k, l)-1) / n)

print(f"연관계수: {round(V, 3)}")

연관계수: 0.408


In [19]:
from scipy.stats.contingency import association

V2 = association(table.values, method='cramer')
print(f"연관계수: {round(V2, 3)}")

연관계수: 0.408


## 연습문제

#### 1.
- 어느 고등학교에서 임의로 추출한 9명의 학생들의 수학과 영어 성적이 다음과 같다.
- 다음 학생들의 성적에 대한 피어슨 상관계수와 스피어만 순위상관계수를 구하고, 각 상관계수가 통계적으로 유의한지 여부를 확인하시오. (유의수준 0.05)
- 또한, 해당 피어슨 상관계수의 신뢰수준 90%의 신뢰구간을 계산하시오.

In [20]:
math = [96, 93, 63, 89, 85, 84, 66, 62, 90]
eng = [98, 90, 74, 84, 69, 69, 73, 61, 98]

from scipy.stats import pearsonr, spearmanr
pr, p1 = pearsonr(math, eng)
sr, p2 = spearmanr(math, eng)

# 두 상관계수 모두 유의확률이 유의수준 0.05보다 작기 때문에 통계적으로 유의하다.
print(f"피어슨   상관계수: {round(pr, 2)} (p-value {round(p1, 2)})")
print(f"스피어만 상관계수: {round(sr, 2)} (p-value {round(p2, 2)})")

피어슨   상관계수: 0.75 (p-value 0.02)
스피어만 상관계수: 0.82 (p-value 0.01)


In [21]:
# 피어슨 상관계수의 신뢰구간 계산

from scipy.stats import norm
conf_a = 0.1 # 신뢰수준 90%
conf_z = norm.ppf(1 - conf_a / 2)

r1 = pr # 위에서 계산한 표본상관계수 pr을 사용
n = 9 # 표본의 개수
a = 1/2 * np.log((1+r1) / (1-r1)) - conf_z / np.sqrt(n-3)
b = 1/2 * np.log((1+r1) / (1-r1)) + conf_z / np.sqrt(n-3)

conf1 = (np.exp(2*a)-1) / (np.exp(2*a)+1)
conf2 = (np.exp(2*b)-1) / (np.exp(2*b)+1)
print(f"구간 추정량: {round(conf1, 3)} ~ {round(conf2, 3)}".format(conf1, conf2))

구간 추정량: 0.283 ~ 0.927


#### 2.
- 두 변수 X와 Y의 표준편차는 각각 4, 5이고 공분산이 4인 경우, 두 변수의 상관계수를 구하시오.

In [22]:
Xs, Ys = 4, 5
cov = 4
cov / (Xs * Ys)

0.2